# Best Results

In [ ]:
import numpy as np
from pathlib import Path
import pandas as pd

path_results = '/content/drive/MyDrive/USP/Doctorate/Research/Articles/Early Fusion For One-class Learning On Heterogeneous Graphs/resultados/GRAPHSAGE/'

basepath = Path(path_results)
datasets = basepath.iterdir()

dict_ = {}

for file_ in datasets:
  if file_.is_file():

    file_name = file_.name

    parts = file_name.split('_')

    dataset = parts[0]

    # if dataset == 'rec':
    #   if parts[1] == 'sys-gae':
    #     dataset = 'rec_sys-gae'
    #   else:
    #     dataset = 'rec_sys'
    #   method = parts[2]
    #else:
    method = parts[1]

    df = pd.read_csv(path_results + file_name, sep=';')

    method = method.replace('.csv', '')

    best_f1 = max(df['macro avg_f1-score-mean'])

    std = df[df['macro avg_f1-score-mean'] == best_f1]['macro avg_f1-score-std'].iloc[0]

    if method not in dict_:
      dict_[method] = {}
      dict_[method+'-std'] = {}
      dict_[method][dataset] = best_f1
      dict_[method+'-std'][dataset] = std
    else:
      dict_[method][dataset] = best_f1
      dict_[method+'-std'][dataset] = std

In [ ]:
pd.DataFrame(dict_)

,sub,sub-std,multiply,multiply-std,min,min-std,sum,sum-std,max,max-std,concatenate,concatenate-std,avg,avg-std,without,without-std
fakenews-sage,0.936082,0.004751,0.943000,0.004769,0.922381,0.004868,0.929154,0.004144,0.925884,0.006172,0.923182,0.006943,0.929154,0.004144,0.924099,0.004495
music-sage,0.572280,0.021499,0.561541,0.007613,0.552764,0.010228,0.580712,0.022764,0.580137,0.013165,0.574639,0.018054,0.581371,0.022067,0.556612,0.016511
rec-sys-sage,0.588168,0.007596,0.623043,0.012463,0.602681,0.023526,0.641889,0.015663,0.571563,0.026266,0.612299,0.003365,0.640847,0.014533,0.650495,0.005982
event-sage,0.958817,0.050439,1.000000,0.000000,0.979408,0.041183,0.979408,0.041183,0.979408,0.041183,0.979408,0.041183,0.979408,0.041183,0.979408,0.041183


In [ ]:
pd.DataFrame(dict_).to_csv('webmedia-sage.csv')

# TSNE

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
import pandas as pd

PATH_TO_GAE_GRAPHS = "/content/drive/MyDrive/USP/Doctorate/Research/Articles/Early Fusion For One-class Learning On Heterogeneous Graphs/grafos_gae"

In [ ]:
def plot(representations, classes):

  X_embedded = TSNE(n_components=2, learning_rate='auto', init='random', random_state=81, perplexity=2).fit_transform(representations)

  df1 = pd.DataFrame(X_embedded)

  df1['Class'] = classes

  g = sns.scatterplot(x=0,y=1,data=df1, hue="Class", legend=True)
  g.set(xlabel=None)
  g.set(ylabel=None)
  plt.show()

In [ ]:
import os
import pickle as pkl

def plot_tsne(value_table, operator, dataset, gnn):

  name = 'gae_features'

  pickle = os.listdir(f"{PATH_TO_GAE_GRAPHS}/{dataset}/{gnn}/")[value_table]

  with open(f"{PATH_TO_GAE_GRAPHS}/{dataset}/{gnn}/{pickle}", "rb") as file:
    G = pkl.load(file)

  if operator == 'without':
    reps_without = []
    classes = []
    for node in G.nodes():
      if type(node) == int:
        reps_without.append(G.nodes[node][name])
        classes.append(G.nodes[node]['label'])

    plot(np.array(reps_without), classes)
  else:
    x = []
    classes = []
    for node in G.nodes():
      if type(node) == int:
        rep = G.nodes[node][name]
        classes.append(G.nodes[node]['label'])
        reps = []
        for n in G.neighbors(node):
          reps.append(G.nodes[n][name])

        rep_o = np.mean(reps, axis=0)

        new_rep=None
        if operator == 'concatenate':
          new_rep = np.concatenate([rep, rep_o])
        elif operator == 'sum':
          new_rep = np.sum([rep, rep_o], axis=0)
        elif operator == 'sub':
          new_rep = np.subtract(rep, rep_o)
        elif operator == 'avg':
          new_rep = np.mean([rep, rep_o], axis=0)
        elif operator == 'min':
          new_rep = np.min([rep, rep_o], axis=0)
        elif operator == 'max':
          new_rep = np.max([rep, rep_o], axis=0)
        elif operator == 'multiply':
          new_rep = np.multiply(rep, rep_o)

        x.append(new_rep)

    plot(np.array(x), classes)

In [16]:
for v in range(48):
  plot_tsne(v, 'without', 'fakenews', 'GRAPHSAGE')

Output hidden; open in https://colab.research.google.com to view.

In [17]:
for v in range(48):
  plot_tsne(v, 'concatenate', 'fakenews', 'GRAPHSAGE')

Output hidden; open in https://colab.research.google.com to view.

In [18]:
for v in range(48):
  plot_tsne(v, 'sum', 'fakenews', 'GRAPHSAGE')

Output hidden; open in https://colab.research.google.com to view.

In [19]:
for v in range(48):
  plot_tsne(v, 'sub', 'fakenews', 'GRAPHSAGE')

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for v in range(48):
  plot_tsne(v, 'avg', 'fakenews', 'GRAPHSAGE')

In [ ]:
for v in range(48):
  plot_tsne(v, 'min', 'fakenews', 'GAT')

Output hidden; open in https://colab.research.google.com to view.

In [14]:
for v in range(48):
  plot_tsne(v, 'max', 'fakenews', 'GAT')

In [15]:
for v in range(48):
  plot_tsne(v, 'multiply', 'fakenews', 'GAT')

Output hidden; open in https://colab.research.google.com to view.